### Some example code demonstrating some super() behaviour


In [48]:
class A():
    def __init__(self):
        print("in A __init__")
        print("self's class is:", self.__class__)
        s = super().__init__()

class B():
    def __init__(self):
        print("in B.__init__")
        s = super().__init__()

class C():
    def __init__(self):
        print("in C.__init__")
        s = super().__init__()

class D(C, B, A):
    def __init__(self):
        print("self's class is:", self.__class__)
        super().__init__()
        

In [49]:
print("D's mro:", D.__mro__)
d = D()

D's mro: (<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
self's class is: <class '__main__.D'>
in C.__init__
in B.__init__
in A __init__
self's class is: <class '__main__.D'>


## super's parameters

To do its thing, super() needs to know two things:

``super(type, obj)``

It needs to know that type (class) that you want the super-classes of, AND it needs to know the actual object instance at the time it is called.

python3 fills these in for you at run time, but in python2, you needed to specify them:

```
class A(object):
    def __init__(self):
        super(A, self).__init__()
```

But why do you need BOTH `A` and `self`? -- isn't `self` an instance of `A`?

Not neccesarily -- if A's method is being called from a subclass, then `self` will be an instance of the subclass. `super()` requires that the object be an instance of the class (or a subclass).

This distiction will come up later....

Again, py3 takes care of this for you, though you CAN still spell it out.

In [50]:
s_c = super(C, d)
print(s_c)

<super: <class 'C'>, <D object>>


This works because `d` is a `D` object, which is a subclass of `C`.

In [53]:
c = C()

in C.__init__


In [54]:
super(D, c)

TypeError: super(type, obj): obj must be an instance or subtype of type

But this fails: `C` is NOT a subclass of `D`

In [55]:
s_a = super(A, d)
print(s_a)

<super: <class 'A'>, <D object>>


In [56]:
s_b = super(B, d)
print(s_b)

<super: <class 'B'>, <D object>>


In [58]:
print(D.__mro__)

(<class '__main__.D'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)


## Use super() everywhere.

In [59]:
# without super()

class A():
    def this(self):
        print("in A.this")

class B():
    def this(self):
        print("in B.this")

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)
        B.this(self)

In [60]:
print("Running without super()")
c = C()
c.this()

Running without super()
in C.this
in A.this
in B.this


C's `this` explicitly called both A and B's methods -- so they all get called.

### Using super in just C:

In [61]:
class A():
    def this(self):
        print("in A.this")

class B(A):
    def this(self):
        print("in B.this")

class C(B):
    def this(self):
        print("in C.this")
        super().this()

In [62]:
print(C.__mro__)
c = C()
c.this()

(<class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
in C.this
in B.this


**Note:**  `A.this` did NOT get called!

Even though it is in in the MRO.

Python stopped when it found the method in B.

### Using super everywhere:

In [64]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()
class C(A,B):
    def this(self):
        print("in C.this")
        super().this()

In [65]:
c = C()
c.this()
print(Base.__mro__)

in C.this
in A.this
in B.this
(<class '__main__.Base'>, <class 'object'>)


Now both A and B's methods get called -- probably what you want.

But if you don't want both called -- better to just be Explicit, rather than use super():

In [66]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)

In [67]:
c = C()
c.this()

in C.this
in A.this
in B.this


**Whoa** -- A and B's method DID get called! -- why?

In [68]:
A.__mro__

(__main__.A, __main__.Base, object)

B is not there.

But:

In [69]:
class Base():
    def this(self):
        pass # just so there is a base that has the method

class A(Base):
    def this(self):
        print("in A.this")
        print("self's class:", self.__class__)
        super().this()

class B(Base):
    def this(self):
        print("in B.this")
        super().this()

class C(A,B):
    def this(self):
        print("in C.this")
        A.this(self)

In [70]:
c = C()
c.this()
print(C.__mro__)

in C.this
in A.this
self's class: <class '__main__.C'>
in B.this
(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)


Remember, `super()` is dynamic -- what it calls is determined at run time.

That's how it knows to call ``B``'s method too. Which is why we say that using `super()` is *part* of the interface of the class.